# Streaming Heavy-Hitters: Implementations of Algorithms 3.1, 3.2, 3.3

このノートブックでは、提示画像の擬似コードを **Python** で実装します。

- **Algorithm 3.1**: Misra–Gries（k-1 カウンタ方式）
- **Algorithm 3.2**: Lossy Counting（Δ=⌊n/k⌋ を用いたバケット削除）
- **Algorithm 3.3**: SpaceSaving（最小カウンタの置換）

いずれもストリーム中の **頻出要素（heavy hitters）** を近似的に追跡するアルゴリズムです。

In [ ]:
from __future__ import annotations
from typing import Dict, Iterable, List, Tuple, Any

def top_k_from_counts(counts: Dict[Any, int], k: int) -> List[Tuple[Any, int]]:
    """Return top-k pairs sorted by count desc, then key asc for stability."""
    return sorted(counts.items(), key=lambda x: (-x[1], x[0]))[:k]

## Algorithm 3.1 — Misra–Gries
擬似コード:
- `n ← n + 1`
- `if i ∈ T` なら `c_i ← c_i + 1`
- そうでなく `|T| < k-1` なら `T ← T ∪ {i}; c_i ← 1`
- それ以外なら、全 `j ∈ T` について `c_j ← c_j - 1` を行い、`c_j = 0` は削除

性質: どの時点でもテーブルは最大 `k-1` 件。真の出現回数 `f(i)` と推定 `c_i` の誤差は **高々 ⌊n/k⌋**。

In [ ]:
class MisraGries:
    """Algorithm 3.1 (Misra–Gries) for heavy hitters with parameter k."""
    def __init__(self, k: int):
        assert k >= 2, "k must be >= 2"
        self.k = k
        self.n = 0
        self.T: Dict[Any, int] = {}  # item -> c_i

    def update(self, item: Any):
        self.n += 1
        if item in self.T:
            self.T[item] += 1
            return
        if len(self.T) < self.k - 1:
            self.T[item] = 1
            return
        # decrement all, drop zeros
        to_del = []
        for j in list(self.T.keys()):
            self.T[j] -= 1
            if self.T[j] == 0:
                to_del.append(j)
        for j in to_del:
            del self.T[j]

    def consume(self, stream: Iterable[Any]):
        for x in stream:
            self.update(x)
        return self

    def candidates(self) -> Dict[Any, int]:
        return dict(self.T)

    def topk(self, k: int | None = None) -> List[Tuple[Any, int]]:
        return top_k_from_counts(self.T, k or len(self.T))

## Algorithm 3.2 — Lossy Counting（Δ=⌊n/k⌋ を用いた一括削除）
擬似コード（画像の記法を Python に忠実移植）:
- 追加の変数 `Δ` を持つ（初期値 0）
- 各到着で `n ← n + 1`。`i ∈ T` なら `c_i ← c_i + 1`、さもなくば `T ← T ∪ {i}; c_i ← 1 + Δ`
- `if ⌊n/k⌋ ≠ Δ` なら `Δ ← ⌊n/k⌋` とし、すべての `j ∈ T` について `if c_j < Δ: remove j`。

性質: 誤差は最大 `n/k`。容量はおおよそ `O(k log n)` に抑えられます（経験則）。

In [ ]:
class LossyCounting:
    """Algorithm 3.2 (Manku–Motwani style lossy counting) with bucket Δ=floor(n/k)."""
    def __init__(self, k: int):
        assert k >= 2, "k must be >= 2"
        self.k = k
        self.n = 0
        self.delta = 0
        self.T: Dict[Any, int] = {}

    def _maybe_prune(self):
        new_delta = self.n // self.k
        if new_delta != self.delta:
            self.delta = new_delta
            # prune entries whose c_j < Δ
            to_del = [j for j, c in list(self.T.items()) if c < self.delta]
            for j in to_del:
                del self.T[j]

    def update(self, item: Any):
        self.n += 1
        if item in self.T:
            self.T[item] += 1
        else:
            self.T[item] = 1 + self.delta
        self._maybe_prune()

    def consume(self, stream: Iterable[Any]):
        for x in stream:
            self.update(x)
        return self

    def candidates(self) -> Dict[Any, int]:
        return dict(self.T)

    def topk(self, k: int | None = None) -> List[Tuple[Any, int]]:
        return top_k_from_counts(self.T, k or len(self.T))

## Algorithm 3.3 — SpaceSaving（最小カウンタの置換）
擬似コード:
- `i ∈ T` なら `c_i ← c_i + 1`
- そうでなく `|T| < k` なら `T ← T ∪ {i}; c_i ← 1`
- それ以外なら `j ← argmin_{j∈T} c_j` を選び、`c_i ← c_j + 1` として `i` で置換

性質: 容量は常に **k**。`f(i)` の過小評価は起きず、誤差は最大で **最小カウンタ値**。

In [ ]:
class SpaceSaving:
    """Algorithm 3.3 (SpaceSaving) with exactly k counters."""
    def __init__(self, k: int):
        assert k >= 1, "k must be >= 1"
        self.k = k
        self.n = 0
        self.T: Dict[Any, int] = {}

    def update(self, item: Any):
        self.n += 1
        if item in self.T:
            self.T[item] += 1
            return
        if len(self.T) < self.k:
            self.T[item] = 1
            return
        # replace the minimum counter entry
        j, cj = min(self.T.items(), key=lambda x: x[1])
        del self.T[j]
        self.T[item] = cj + 1

    def consume(self, stream: Iterable[Any]):
        for x in stream:
            self.update(x)
        return self

    def candidates(self) -> Dict[Any, int]:
        return dict(self.T)

    def topk(self, k: int | None = None) -> List[Tuple[Any, int]]:
        return top_k_from_counts(self.T, k or len(self.T))

## Quick Demo
同じストリームに対して 3 つのアルゴリズムを走らせ、上位候補を比較します。

In [ ]:
import random
random.seed(0)

# 合成ストリーム（'A' と 'B' が heavy）
stream = ['A']*120 + ['B']*90 + ['C']*40 + ['D']*35 + ['E']*30 + ['F']*20
random.shuffle(stream)

k = 5
mg = MisraGries(k).consume(stream)
lc = LossyCounting(k).consume(stream)
ss = SpaceSaving(k).consume(stream)

print("n =", len(stream))
print("Misra–Gries:", mg.topk())
print("Lossy Counting:", lc.topk())
print("SpaceSaving:", ss.topk())